# PNADC Exploration (NPV-ready)

Este notebook normaliza o fluxo: (1) busca IPCA do BCB, (2) ajusta rendimentos para valor presente (mês mais recente disponível), (3) adiciona coluna em salários mínimos, (4) persiste Parquet/CSV NPV e (5) conecta DuckDB usando o DataFrame ajustado.


In [ ]:
# Imports & display configuration
import pandas as pd
import numpy as np
import duckdb
from urllib.request import urlopen, Request
import json
from pathlib import Path
from IPython.display import display
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 160)


## NPV Setup (IPCA BCB) — deflate VD4020 ao valor presente

- Busca IPCA mensal (variação %) no BCB (SGS 433).
- Compõe índice encadeado e rebaseia no mês-alvo mais recente.
- Calcula `factor_to_target` = índice(alvo) / índice(mês). A base cancela.


In [ ]:
INCOME_COL = 'VD4020__rendim_efetivo_qq_trabalho'
MIN_WAGE = 1518.0  # salário mínimo para converter em 'salários mínimos'

def fetch_ipca_bcb_variation(series: int = 433) -> pd.DataFrame:
    url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{series}/dados?formato=json'
    req = Request(url, headers={'User-Agent': 'pnad-npv/1.0'})
    with urlopen(req, timeout=60) as resp:
        data = resp.read().decode('utf-8')
    items = json.loads(data)
    out = []
    for it in items:
        d = str(it.get('data', ''))
        parts = d.split('/')
        if len(parts) == 2:
            m, y = int(parts[0]), int(parts[1])
        else:
            m, y = int(parts[1]), int(parts[2])
        val = float(str(it.get('valor', '')).replace(',', '.'))
        out.append((f'{y}-{m:02d}', val))
    df = pd.DataFrame(out, columns=['ym', 'pct_month']).sort_values('ym').reset_index(drop=True)
    return df

def build_index_and_factors(df_pct: pd.DataFrame) -> pd.DataFrame:
    idx = (1.0 + df_pct['pct_month'] / 100.0).cumprod()
    out = pd.DataFrame({'ym': df_pct['ym'], 'index': idx})
    target_ym = out['ym'].iloc[-1]
    target_idx = float(out.loc[out['ym'] == target_ym, 'index'].iloc[0])
    out['factor_to_target'] = target_idx / out['index']
    return out, target_ym

ipca_pct = fetch_ipca_bcb_variation()
ipca_idx, TARGET_YM = build_index_and_factors(ipca_pct)
display({'target_month': TARGET_YM})


## Load data, apply NPV adjustments, save outputs, and setup DuckDB

- Resolve `data/base_labeled.{parquet,csv}` subindo diretórios.
- Deriva `ym` de Ano/Trimestre (último mês do trimestre).
- Faz merge com fatores e cria colunas deflacionadas e em salários mínimos.
- Salva `data/base_labeled_npv.{parquet,csv}`.
- Registra `df` como view `base` no DuckDB.


In [ ]:
def find_in_data(fn: str) -> Path | None:
    for base in [Path.cwd()] + list(Path.cwd().parents):
        cand = base / 'data' / fn
        if cand.exists():
            return cand
    return None

# Load data
parquet_file = find_in_data('base_labeled.parquet')
csv_file = find_in_data('base_labeled.csv')
if parquet_file:
    df = pd.read_parquet(parquet_file)
elif csv_file:
    df = pd.read_csv(csv_file)
else:
    raise FileNotFoundError('Missing base_labeled.{parquet,csv} under data/')
display({'resolved_parquet': str(parquet_file) if parquet_file else None, 'resolved_csv': str(csv_file) if csv_file else None})

# Create year-month column from Ano and Trimestre
Q2M = {1: '03', 2: '06', 3: '09', 4: '12'}
def to_int(x):
    try:
        return int(str(x).strip())
    except Exception:
        return None
year_col = next((c for c in df.columns if c.startswith('Ano__')), 'Ano__ano_de_referncia')
quarter_col = next((c for c in df.columns if c.startswith('Trimestre__')), 'Trimestre__trimestre_de_referncia')
df['ym'] = df[year_col].map(lambda x: str(x).split('.')[0]) + '-' + df[quarter_col].map(lambda x: Q2M.get(to_int(x), '12'))

# Apply NPV adjustments
df = df.merge(ipca_idx[['ym','factor_to_target']], on='ym', how='left')
npv_col = f"{INCOME_COL}_{TARGET_YM.replace('-', '')}"
mw_col = f'{INCOME_COL}_mw'
df[npv_col] = pd.to_numeric(df[INCOME_COL], errors='coerce') * df['factor_to_target']
df[mw_col] = df[npv_col] / float(MIN_WAGE)
display(df[[INCOME_COL, npv_col, mw_col]].describe(include='all'))

# Save NPV-adjusted data
out_dir = (parquet_file or csv_file).parent
npv_parquet = out_dir / 'base_labeled_npv.parquet'
df.to_parquet(npv_parquet, index=False)
npv_csv = out_dir / 'base_labeled_npv.csv'
try:
    df.to_csv(npv_csv, index=False)
except Exception:
    pass
display({'saved_parquet': str(npv_parquet), 'saved_csv': str(npv_csv)})

# Setup DuckDB connection
con = duckdb.connect(database=':memory:')
try:
    con.register('base', df)
    source = 'df'
except Exception:
    con.sql(f"CREATE OR REPLACE VIEW base AS SELECT * FROM read_parquet('{str(npv_parquet).replace(chr(39),chr(39)+chr(39))}')") 
    source = str(npv_parquet)
display({'duck_source': source})
con.sql('select count(*) as rows from base').df()


## Individual Income Analysis - Minimum Wage Bands

Análise da distribuição de renda individual em faixas de salários mínimos.

In [ ]:
# Individual Income Band Analysis - Version 1 (0-2, 2-5, 5+ MW)
mw_col = f'{INCOME_COL}_mw'

# Filter valid income data (exclude nulls and zeros)
df_income = df[df[mw_col].notna() & (df[mw_col] > 0)].copy()

# Create income bands
def categorize_income_v1(mw):
    if mw < 2:
        return '0-2 MW'
    elif mw < 5:
        return '2-5 MW'
    else:
        return '5+ MW'

df_income['income_band_v1'] = df_income[mw_col].apply(categorize_income_v1)

# Calculate distribution
income_dist_v1 = df_income['income_band_v1'].value_counts().sort_index()
income_pct_v1 = (income_dist_v1 / income_dist_v1.sum() * 100).round(2)

# Display results
display({
    'Total people with income': len(df_income),
    'Income bands (0-2, 2-5, 5+ MW)': pd.DataFrame({
        'Count': income_dist_v1,
        'Percentage': income_pct_v1
    })
})

# Summary statistics by band
display(df_income.groupby('income_band_v1')[mw_col].describe().round(2))

In [ ]:
# Individual Income Band Analysis - Version 2 (0-2, 2-5, 5-10, 10+ MW)
def categorize_income_v2(mw):
    if mw < 2:
        return '0-2 MW'
    elif mw < 5:
        return '2-5 MW'
    elif mw < 10:
        return '5-10 MW'
    else:
        return '10+ MW'

df_income['income_band_v2'] = df_income[mw_col].apply(categorize_income_v2)

# Calculate distribution
income_dist_v2 = df_income['income_band_v2'].value_counts().sort_index()
income_pct_v2 = (income_dist_v2 / income_dist_v2.sum() * 100).round(2)

# Display results
display({
    'Income bands (0-2, 2-5, 5-10, 10+ MW)': pd.DataFrame({
        'Count': income_dist_v2,
        'Percentage': income_pct_v2
    })
})

# Summary statistics by band
display(df_income.groupby('income_band_v2')[mw_col].describe().round(2))

## Household Income Analysis

Agregação de renda por domicílio e análise das faixas de renda familiar.

In [ ]:
# Household Income Aggregation
# Create household ID (dom_id)
df['dom_id'] = (df['Ano__ano_de_referncia'].astype(str) + '_' +
                df['Trimestre__trimestre_de_referncia'].astype(str) + '_' +
                df['UPA__unidade_primria_de_amostragem'].astype(str) + '_' +
                df['V1008__nmero_de_seleo_do_domiclio'].astype(str))

# Aggregate income by household
household_income = df.groupby('dom_id').agg({
    mw_col: 'sum',  # Total household income in MW
    npv_col: 'sum',  # Total household income NPV-adjusted
    'VD2003__nmero_de_componentes_do_domic': 'first',  # Number of household members
    'UF_label': 'first',
    'Capital_label': 'first',
    'RM_RIDE_label': 'first'
}).rename(columns={
    mw_col: 'household_income_mw',
    npv_col: 'household_income_npv',
    'VD2003__nmero_de_componentes_do_domic': 'household_size'
})

# Calculate per capita income
household_income['per_capita_income_mw'] = household_income['household_income_mw'] / household_income['household_size']

# Filter valid households
household_income = household_income[household_income['household_income_mw'].notna() & 
                                    (household_income['household_income_mw'] > 0)]

display({
    'Total households': len(household_income),
    'Average household size': household_income['household_size'].mean().round(2),
    'Household income statistics (MW)': household_income['household_income_mw'].describe().round(2)
})

In [ ]:
# Household Income Bands Analysis
def categorize_household_income(mw):
    if mw < 2:
        return '0-2 MW'
    elif mw < 5:
        return '2-5 MW'
    elif mw < 10:
        return '5-10 MW'
    else:
        return '10+ MW'

household_income['income_band'] = household_income['household_income_mw'].apply(categorize_household_income)

# Calculate distribution
household_dist = household_income['income_band'].value_counts().sort_index()
household_pct = (household_dist / household_dist.sum() * 100).round(2)

display({
    'Household income bands': pd.DataFrame({
        'Count': household_dist,
        'Percentage': household_pct
    })
})

# Per capita income bands
household_income['per_capita_band'] = household_income['per_capita_income_mw'].apply(categorize_household_income)
per_capita_dist = household_income['per_capita_band'].value_counts().sort_index()
per_capita_pct = (per_capita_dist / per_capita_dist.sum() * 100).round(2)

display({
    'Per capita income bands': pd.DataFrame({
        'Count': per_capita_dist,
        'Percentage': per_capita_pct
    })
})

## Income Distribution Visualizations

Histogramas e gráficos da distribuição de renda.

In [ ]:
# Import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 10)

# Create subplots for income distributions
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Histogram of individual income in MW
ax1 = axes[0, 0]
df_income_plot = df_income[df_income[mw_col] <= 20]  # Limit for better visualization
ax1.hist(df_income_plot[mw_col], bins=50, edgecolor='black', alpha=0.7)
ax1.set_xlabel('Income (Minimum Wages)')
ax1.set_ylabel('Frequency')
ax1.set_title('Individual Income Distribution (MW)')
ax1.axvline(x=1, color='r', linestyle='--', label='1 MW')
ax1.axvline(x=2, color='g', linestyle='--', label='2 MW')
ax1.axvline(x=5, color='b', linestyle='--', label='5 MW')
ax1.legend()

# 2. Histogram of NPV-adjusted income
ax2 = axes[0, 1]
df_npv_plot = df_income[df_income[npv_col] <= 30000]  # Limit for better visualization
ax2.hist(df_npv_plot[npv_col], bins=50, edgecolor='black', alpha=0.7, color='orange')
ax2.set_xlabel('Income (R$ - Jul/2025)')
ax2.set_ylabel('Frequency')
ax2.set_title('Individual Income Distribution (NPV-adjusted)')
ax2.axvline(x=MIN_WAGE, color='r', linestyle='--', label=f'1 MW (R$ {MIN_WAGE})')
ax2.axvline(x=MIN_WAGE*2, color='g', linestyle='--', label=f'2 MW')
ax2.axvline(x=MIN_WAGE*5, color='b', linestyle='--', label=f'5 MW')
ax2.legend()

# 3. Bar chart of income bands (individual)
ax3 = axes[0, 2]
income_pct_v2.plot(kind='bar', ax=ax3, color=['#ff9999', '#66b3ff', '#99ff99', '#ffcc99'])
ax3.set_xlabel('Income Band')
ax3.set_ylabel('Percentage (%)')
ax3.set_title('Individual Income Distribution by Bands')
ax3.set_xticklabels(ax3.get_xticklabels(), rotation=45)
for i, v in enumerate(income_pct_v2):
    ax3.text(i, v + 0.5, f'{v:.1f}%', ha='center')

# 4. Household income histogram
ax4 = axes[1, 0]
household_plot = household_income[household_income['household_income_mw'] <= 30]
ax4.hist(household_plot['household_income_mw'], bins=50, edgecolor='black', alpha=0.7, color='green')
ax4.set_xlabel('Household Income (MW)')
ax4.set_ylabel('Frequency')
ax4.set_title('Household Income Distribution')
ax4.axvline(x=2, color='r', linestyle='--', label='2 MW')
ax4.axvline(x=5, color='g', linestyle='--', label='5 MW')
ax4.axvline(x=10, color='b', linestyle='--', label='10 MW')
ax4.legend()

# 5. Per capita income histogram
ax5 = axes[1, 1]
per_capita_plot = household_income[household_income['per_capita_income_mw'] <= 10]
ax5.hist(per_capita_plot['per_capita_income_mw'], bins=50, edgecolor='black', alpha=0.7, color='purple')
ax5.set_xlabel('Per Capita Income (MW)')
ax5.set_ylabel('Frequency')
ax5.set_title('Per Capita Income Distribution')
ax5.axvline(x=0.5, color='r', linestyle='--', label='0.5 MW')
ax5.axvline(x=1, color='g', linestyle='--', label='1 MW')
ax5.axvline(x=2, color='b', linestyle='--', label='2 MW')
ax5.legend()

# 6. Pie chart of household income bands
ax6 = axes[1, 2]
colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99']
wedges, texts, autotexts = ax6.pie(household_pct, labels=household_pct.index, colors=colors, 
                                    autopct='%1.1f%%', startangle=90)
ax6.set_title('Household Income Distribution by Bands')

plt.tight_layout()
plt.show()

# Summary statistics
display({
    'Income Distribution Summary': {
        'Individual median (MW)': df_income[mw_col].median().round(2),
        'Individual mean (MW)': df_income[mw_col].mean().round(2),
        'Household median (MW)': household_income['household_income_mw'].median().round(2),
        'Household mean (MW)': household_income['household_income_mw'].mean().round(2),
        'Per capita median (MW)': household_income['per_capita_income_mw'].median().round(2),
        'Per capita mean (MW)': household_income['per_capita_income_mw'].mean().round(2)
    }
})

## Geographic and Demographic Analysis

Análise de renda por região, estado, e características demográficas.

In [ ]:
# Geographic Analysis - Income by State
state_income = df_income.groupby('UF_label').agg({
    mw_col: ['mean', 'median', 'std', 'count']
}).round(2)
state_income.columns = ['Mean MW', 'Median MW', 'Std MW', 'Count']
state_income = state_income.sort_values('Median MW', ascending=False)

# Top 10 states by median income
display({
    'Top 10 States by Median Income (MW)': state_income.head(10)
})

# Income by Capital vs Non-Capital
capital_income = df_income.groupby('Capital_label').agg({
    mw_col: ['mean', 'median', 'count']
}).round(2)
display({
    'Income by Capital Status': capital_income
})

# Demographic Analysis
# By Sex
sex_income = df_income.groupby('V2007_label').agg({
    mw_col: ['mean', 'median', 'count']
}).round(2)
sex_income.columns = ['Mean MW', 'Median MW', 'Count']

# By Age Groups
df_income['age_group'] = pd.cut(df_income['V2009__idade_na_data_de_referncia'], 
                                bins=[0, 18, 25, 35, 45, 55, 65, 100],
                                labels=['<18', '18-25', '26-35', '36-45', '46-55', '56-65', '65+'])
age_income = df_income.groupby('age_group').agg({
    mw_col: ['mean', 'median', 'count']
}).round(2)
age_income.columns = ['Mean MW', 'Median MW', 'Count']

# By Race
race_income = df_income.groupby('V2010_label').agg({
    mw_col: ['mean', 'median', 'count']
}).round(2)
race_income.columns = ['Mean MW', 'Median MW', 'Count']

display({
    'Income by Sex': sex_income,
    'Income by Age Group': age_income,
    'Income by Race': race_income
})

## Predictive Model for Income Estimation

Modelo de machine learning para prever renda baseado em características demográficas e geográficas.

In [ ]:
# Data Preparation for Predictive Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Select features for the model
feature_columns = [
    'UF__unidade_da_federao', 'Capital__municpio_da_capital',
    'V2007__sexo', 'V2009__idade_na_data_de_referncia', 'V2010__cor_ou_raa',
    'V3001__sabe_ler_e_escrever', 'V3009A__curso_mais_elevado_que_frequentou',
    'VD2003__nmero_de_componentes_do_domic'
]

# Prepare dataset
model_df = df_income[feature_columns + [mw_col]].dropna()

# Encode categorical variables
label_encoders = {}
categorical_cols = ['UF__unidade_da_federao', 'Capital__municpio_da_capital', 
                   'V2007__sexo', 'V2010__cor_ou_raa', 'V3001__sabe_ler_e_escrever',
                   'V3009A__curso_mais_elevado_que_frequentou']

for col in categorical_cols:
    le = LabelEncoder()
    model_df[col] = le.fit_transform(model_df[col].astype(str))
    label_encoders[col] = le

# Split features and target
X = model_df[feature_columns]
y = model_df[mw_col]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

display({
    'Dataset shape': model_df.shape,
    'Training set': X_train.shape,
    'Test set': X_test.shape,
    'Target distribution': y.describe().round(2)
})

In [ ]:
# Model Training and Evaluation
# Random Forest Model
rf_model = RandomForestRegressor(n_estimators=100, max_depth=20, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

# Gradient Boosting Model
gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=10, random_state=42)
gb_model.fit(X_train, y_train)
gb_pred = gb_model.predict(X_test)

# Model evaluation
def evaluate_model(y_true, y_pred, model_name):
    return {
        'Model': model_name,
        'R² Score': round(r2_score(y_true, y_pred), 4),
        'MAE': round(mean_absolute_error(y_true, y_pred), 3),
        'RMSE': round(np.sqrt(mean_squared_error(y_true, y_pred)), 3),
        'Median Abs Error': np.median(np.abs(y_true - y_pred)).round(3)
    }

# Compare models
results = pd.DataFrame([
    evaluate_model(y_test, rf_pred, 'Random Forest'),
    evaluate_model(y_test, gb_pred, 'Gradient Boosting')
])

display({
    'Model Performance': results
})

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': feature_columns,
    'RF_Importance': rf_model.feature_importances_,
    'GB_Importance': gb_model.feature_importances_
}).sort_values('RF_Importance', ascending=False)

display({
    'Feature Importance': feature_importance.round(4)
})

# Visualize predictions vs actual
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Random Forest
ax1 = axes[0]
ax1.scatter(y_test, rf_pred, alpha=0.5, s=10)
ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
ax1.set_xlabel('Actual Income (MW)')
ax1.set_ylabel('Predicted Income (MW)')
ax1.set_title(f'Random Forest (R²={r2_score(y_test, rf_pred):.3f})')
ax1.set_xlim(0, 15)
ax1.set_ylim(0, 15)

# Gradient Boosting
ax2 = axes[1]
ax2.scatter(y_test, gb_pred, alpha=0.5, s=10, color='green')
ax2.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
ax2.set_xlabel('Actual Income (MW)')
ax2.set_ylabel('Predicted Income (MW)')
ax2.set_title(f'Gradient Boosting (R²={r2_score(y_test, gb_pred):.3f})')
ax2.set_xlim(0, 15)
ax2.set_ylim(0, 15)

plt.tight_layout()
plt.show()

# Advanced Statistical Analysis

Este seção contém análises estatísticas avançadas para desigualdade de renda, testes estatísticos, mobilidade de renda e análises multivariadas.

In [ ]:
# Import additional libraries for advanced analysis
from scipy import stats
from scipy.stats import mannwhitneyu, kruskal, chi2_contingency, ttest_ind
from scipy.optimize import minimize_scalar
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

print("Advanced statistical analysis libraries loaded successfully!")

## 1. Gini Coefficient and Lorenz Curve Analysis

Cálculo do coeficiente de Gini e construção da curva de Lorenz para análise da desigualdade de renda.

In [ ]:
def calculate_gini(income_array):
    """
    Calculate Gini coefficient for income distribution.
    
    Args:
        income_array: Array of income values
    
    Returns:
        Gini coefficient (0 = perfect equality, 1 = perfect inequality)
    """
    # Remove NaN and negative values
    income = np.array(income_array)
    income = income[~np.isnan(income)]
    income = income[income >= 0]
    
    if len(income) == 0:
        return np.nan
    
    # Sort income array
    income_sorted = np.sort(income)
    n = len(income_sorted)
    
    # Calculate Gini coefficient using the formula:
    # G = (2 * sum(i * y_i)) / (n * sum(y_i)) - (n + 1) / n
    cumsum = np.cumsum(income_sorted)
    gini = (2 * np.sum((np.arange(1, n + 1) * income_sorted))) / (n * np.sum(income_sorted)) - (n + 1) / n
    
    return gini

def calculate_lorenz_curve(income_array):
    """
    Calculate Lorenz curve coordinates.
    
    Args:
        income_array: Array of income values
    
    Returns:
        tuple: (population_cumulative, income_cumulative) for plotting
    """
    # Remove NaN and negative values
    income = np.array(income_array)
    income = income[~np.isnan(income)]
    income = income[income >= 0]
    
    if len(income) == 0:
        return np.array([]), np.array([])
    
    # Sort income
    income_sorted = np.sort(income)
    n = len(income_sorted)
    
    # Calculate cumulative proportions
    population_cumulative = np.arange(1, n + 1) / n
    income_cumulative = np.cumsum(income_sorted) / np.sum(income_sorted)
    
    # Add origin point
    population_cumulative = np.concatenate([[0], population_cumulative])
    income_cumulative = np.concatenate([[0], income_cumulative])
    
    return population_cumulative, income_cumulative

# Calculate Gini coefficient for different income measures
gini_results = {}

# Individual income (MW)
gini_individual_mw = calculate_gini(df_income[mw_col])
gini_results['Individual Income (MW)'] = gini_individual_mw

# Individual income (NPV-adjusted)
gini_individual_npv = calculate_gini(df_income[npv_col])
gini_results['Individual Income (NPV)'] = gini_individual_npv

# Household income (MW)
gini_household_mw = calculate_gini(household_income['household_income_mw'])
gini_results['Household Income (MW)'] = gini_household_mw

# Per capita income (MW)
gini_per_capita_mw = calculate_gini(household_income['per_capita_income_mw'])
gini_results['Per Capita Income (MW)'] = gini_per_capita_mw

# Display Gini coefficients
gini_df = pd.DataFrame(list(gini_results.items()), columns=['Income Type', 'Gini Coefficient'])
gini_df['Gini Coefficient'] = gini_df['Gini Coefficient'].round(4)

display({
    'Gini Coefficients': gini_df
})

print("\nInterpretation:")
print("- Gini = 0: Perfect equality (everyone has same income)")
print("- Gini = 1: Perfect inequality (one person has all income)")
print("- Values 0.25-0.35: Relatively equal")
print("- Values 0.35-0.45: Moderately unequal")
print("- Values > 0.45: Highly unequal")

In [ ]:
# Plot Lorenz Curves
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Individual Income (MW) Lorenz Curve
ax1 = axes[0, 0]
pop_cum, inc_cum = calculate_lorenz_curve(df_income[mw_col])
ax1.plot(pop_cum, inc_cum, 'b-', linewidth=2, label='Lorenz Curve')
ax1.plot([0, 1], [0, 1], 'r--', linewidth=1, label='Line of Equality')
ax1.fill_between(pop_cum, inc_cum, 0, alpha=0.3, color='lightblue')
ax1.set_xlabel('Cumulative Share of Population')
ax1.set_ylabel('Cumulative Share of Income')
ax1.set_title(f'Individual Income Lorenz Curve\n(Gini = {gini_individual_mw:.4f})')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Household Income Lorenz Curve
ax2 = axes[0, 1]
pop_cum_h, inc_cum_h = calculate_lorenz_curve(household_income['household_income_mw'])
ax2.plot(pop_cum_h, inc_cum_h, 'g-', linewidth=2, label='Lorenz Curve')
ax2.plot([0, 1], [0, 1], 'r--', linewidth=1, label='Line of Equality')
ax2.fill_between(pop_cum_h, inc_cum_h, 0, alpha=0.3, color='lightgreen')
ax2.set_xlabel('Cumulative Share of Households')
ax2.set_ylabel('Cumulative Share of Household Income')
ax2.set_title(f'Household Income Lorenz Curve\n(Gini = {gini_household_mw:.4f})')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Per Capita Income Lorenz Curve
ax3 = axes[1, 0]
pop_cum_pc, inc_cum_pc = calculate_lorenz_curve(household_income['per_capita_income_mw'])
ax3.plot(pop_cum_pc, inc_cum_pc, 'm-', linewidth=2, label='Lorenz Curve')
ax3.plot([0, 1], [0, 1], 'r--', linewidth=1, label='Line of Equality')
ax3.fill_between(pop_cum_pc, inc_cum_pc, 0, alpha=0.3, color='lavender')
ax3.set_xlabel('Cumulative Share of Population')
ax3.set_ylabel('Cumulative Share of Per Capita Income')
ax3.set_title(f'Per Capita Income Lorenz Curve\n(Gini = {gini_per_capita_mw:.4f})')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Comparison of all Lorenz curves
ax4 = axes[1, 1]
ax4.plot(pop_cum, inc_cum, 'b-', linewidth=2, label=f'Individual (Gini={gini_individual_mw:.3f})')
ax4.plot(pop_cum_h, inc_cum_h, 'g-', linewidth=2, label=f'Household (Gini={gini_household_mw:.3f})')
ax4.plot(pop_cum_pc, inc_cum_pc, 'm-', linewidth=2, label=f'Per Capita (Gini={gini_per_capita_mw:.3f})')
ax4.plot([0, 1], [0, 1], 'r--', linewidth=1, label='Perfect Equality')
ax4.set_xlabel('Cumulative Share of Population')
ax4.set_ylabel('Cumulative Share of Income')
ax4.set_title('Comparison of Lorenz Curves')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate income shares by deciles
def calculate_decile_shares(income_array):
    """Calculate income share for each decile."""
    income = np.array(income_array)
    income = income[~np.isnan(income)]
    income = income[income >= 0]
    
    if len(income) == 0:
        return np.array([])
    
    # Calculate deciles
    deciles = np.percentile(income, np.arange(10, 101, 10))
    decile_labels = [f'D{i}' for i in range(1, 11)]
    
    # Assign each person to a decile
    decile_assignment = np.digitize(income, deciles)
    
    # Calculate total income for each decile
    total_income = np.sum(income)
    decile_shares = []
    
    for i in range(1, 11):
        decile_income = np.sum(income[decile_assignment == i-1])
        share = decile_income / total_income * 100
        decile_shares.append(share)
    
    return np.array(decile_shares), decile_labels

# Calculate and display decile shares
decile_shares, decile_labels = calculate_decile_shares(df_income[mw_col])

decile_df = pd.DataFrame({
    'Decile': decile_labels,
    'Income Share (%)': decile_shares.round(2)
})

display({
    'Income Distribution by Deciles': decile_df,
    'Bottom 50% share': f"{decile_shares[:5].sum():.1f}%",
    'Top 10% share': f"{decile_shares[-1]:.1f}%",
    'Top 1% analysis': "Use percentile 99 for more detailed top income analysis"
})

# Calculate Palma ratio (top 10% / bottom 40%)
palma_ratio = decile_shares[-1] / decile_shares[:4].sum()
display({
    'Palma Ratio': f"{palma_ratio:.2f} (Top 10% vs Bottom 40%)"
})

## 2. Statistical Tests for Income Distribution Comparisons

Testes estatísticos para comparar distribuições de renda entre diferentes grupos demográficos.

In [ ]:
def perform_income_tests(df, income_col, group_col, group_name):
    """
    Perform comprehensive statistical tests comparing income distributions across groups.
    
    Args:
        df: DataFrame with income and grouping data
        income_col: Name of income column
        group_col: Name of grouping column
        group_name: Descriptive name for the grouping variable
    
    Returns:
        Dictionary with test results
    """
    results = {'group_name': group_name}
    
    # Clean data
    df_clean = df[[income_col, group_col]].dropna()
    df_clean = df_clean[df_clean[income_col] > 0]
    
    # Get unique groups
    groups = df_clean[group_col].unique()
    results['groups'] = groups
    results['group_sizes'] = df_clean[group_col].value_counts().to_dict()
    
    # Descriptive statistics by group
    desc_stats = df_clean.groupby(group_col)[income_col].agg([
        'count', 'mean', 'median', 'std', 'min', 'max'
    ]).round(3)
    results['descriptive_stats'] = desc_stats
    
    # 1. Kruskal-Wallis Test (non-parametric ANOVA)
    group_data = [df_clean[df_clean[group_col] == group][income_col].values for group in groups]
    kw_stat, kw_p = kruskal(*group_data)
    results['kruskal_wallis'] = {
        'statistic': kw_stat,
        'p_value': kw_p,
        'significant': kw_p < 0.05,
        'interpretation': 'Significant differences between groups' if kw_p < 0.05 else 'No significant differences'
    }
    
    # 2. Pairwise Mann-Whitney U tests (if more than 2 groups)
    if len(groups) > 2:
        pairwise_results = {}
        for i, group1 in enumerate(groups):
            for j, group2 in enumerate(groups):
                if i < j:  # Avoid duplicate comparisons
                    data1 = df_clean[df_clean[group_col] == group1][income_col]
                    data2 = df_clean[df_clean[group_col] == group2][income_col]
                    
                    stat, p_val = mannwhitneyu(data1, data2, alternative='two-sided')
                    pairwise_results[f'{group1}_vs_{group2}'] = {
                        'statistic': stat,
                        'p_value': p_val,
                        'significant': p_val < 0.05
                    }
        results['pairwise_mann_whitney'] = pairwise_results
    
    # 3. Effect size (Cohen's d for each pair vs overall mean)
    overall_mean = df_clean[income_col].mean()
    overall_std = df_clean[income_col].std()
    
    effect_sizes = {}
    for group in groups:
        group_mean = df_clean[df_clean[group_col] == group][income_col].mean()
        cohens_d = (group_mean - overall_mean) / overall_std
        effect_sizes[group] = cohens_d
    
    results['effect_sizes'] = effect_sizes
    
    # 4. Levene's test for homogeneity of variance
    levene_stat, levene_p = stats.levene(*group_data)
    results['levene_test'] = {
        'statistic': levene_stat,
        'p_value': levene_p,
        'equal_variances': levene_p > 0.05
    }
    
    return results

# Test 1: Income by Gender
print("=== STATISTICAL TESTS FOR INCOME DISTRIBUTION COMPARISONS ===\n")

gender_tests = perform_income_tests(df_income, mw_col, 'V2007_label', 'Gender')

print("1. INCOME BY GENDER")
print("-" * 50)
print(f"Groups: {gender_tests['groups']}")
print(f"Sample sizes: {gender_tests['group_sizes']}")
print("\nDescriptive Statistics:")
display(gender_tests['descriptive_stats'])

print(f"\nKruskal-Wallis Test:")
print(f"  Statistic: {gender_tests['kruskal_wallis']['statistic']:.4f}")
print(f"  P-value: {gender_tests['kruskal_wallis']['p_value']:.6f}")
print(f"  Result: {gender_tests['kruskal_wallis']['interpretation']}")

if 'pairwise_mann_whitney' in gender_tests:
    print(f"\nPairwise Mann-Whitney U Tests:")
    for comparison, result in gender_tests['pairwise_mann_whitney'].items():
        print(f"  {comparison}: p = {result['p_value']:.6f} {'*' if result['significant'] else ''}")

print(f"\nEffect Sizes (Cohen's d):")
for group, effect in gender_tests['effect_sizes'].items():
    magnitude = 'small' if abs(effect) < 0.5 else 'medium' if abs(effect) < 0.8 else 'large'
    print(f"  {group}: {effect:.3f} ({magnitude})")

print(f"\nLevene's Test for Equal Variances:")
print(f"  P-value: {gender_tests['levene_test']['p_value']:.6f}")
print(f"  Equal variances: {gender_tests['levene_test']['equal_variances']}")

print("\n" + "="*70 + "\n")

In [ ]:
# Test 2: Income by Race
race_tests = perform_income_tests(df_income, mw_col, 'V2010_label', 'Race')

print("2. INCOME BY RACE")
print("-" * 50)
print(f"Groups: {race_tests['groups']}")
print(f"Sample sizes: {race_tests['group_sizes']}")
print("\nDescriptive Statistics:")
display(race_tests['descriptive_stats'])

print(f"\nKruskal-Wallis Test:")
print(f"  Statistic: {race_tests['kruskal_wallis']['statistic']:.4f}")
print(f"  P-value: {race_tests['kruskal_wallis']['p_value']:.6f}")
print(f"  Result: {race_tests['kruskal_wallis']['interpretation']}")

if 'pairwise_mann_whitney' in race_tests:
    print(f"\nPairwise Mann-Whitney U Tests:")
    for comparison, result in race_tests['pairwise_mann_whitney'].items():
        print(f"  {comparison}: p = {result['p_value']:.6f} {'*' if result['significant'] else ''}")

print(f"\nEffect Sizes (Cohen's d):")
for group, effect in race_tests['effect_sizes'].items():
    magnitude = 'small' if abs(effect) < 0.5 else 'medium' if abs(effect) < 0.8 else 'large'
    print(f"  {group}: {effect:.3f} ({magnitude})")

print("\n" + "="*70 + "\n")

# Test 3: Income by Age Group
age_tests = perform_income_tests(df_income, mw_col, 'age_group', 'Age Group')

print("3. INCOME BY AGE GROUP")
print("-" * 50)
print(f"Groups: {age_tests['groups']}")
print(f"Sample sizes: {age_tests['group_sizes']}")
print("\nDescriptive Statistics:")
display(age_tests['descriptive_stats'])

print(f"\nKruskal-Wallis Test:")
print(f"  Statistic: {age_tests['kruskal_wallis']['statistic']:.4f}")
print(f"  P-value: {age_tests['kruskal_wallis']['p_value']:.6f}")
print(f"  Result: {age_tests['kruskal_wallis']['interpretation']}")

print(f"\nEffect Sizes (Cohen's d):")
for group, effect in age_tests['effect_sizes'].items():
    if pd.notna(effect):
        magnitude = 'small' if abs(effect) < 0.5 else 'medium' if abs(effect) < 0.8 else 'large'
        print(f"  {group}: {effect:.3f} ({magnitude})")

print("\n" + "="*70 + "\n")

# Test 4: Income by Region (using top 10 states by sample size)
top_states = df_income['UF_label'].value_counts().head(10).index
df_income_states = df_income[df_income['UF_label'].isin(top_states)]

state_tests = perform_income_tests(df_income_states, mw_col, 'UF_label', 'State (Top 10)')

print("4. INCOME BY STATE (TOP 10 BY SAMPLE SIZE)")
print("-" * 50)
print(f"States analyzed: {list(state_tests['groups'])}")
print(f"Sample sizes: {state_tests['group_sizes']}")

print(f"\nKruskal-Wallis Test:")
print(f"  Statistic: {state_tests['kruskal_wallis']['statistic']:.4f}")
print(f"  P-value: {state_tests['kruskal_wallis']['p_value']:.6f}")
print(f"  Result: {state_tests['kruskal_wallis']['interpretation']}")

# Show descriptive stats for top 5 and bottom 5 states by median income
state_medians = state_tests['descriptive_stats']['median'].sort_values(ascending=False)
print(f"\nTop 5 States by Median Income (MW):")
for state, median in state_medians.head(5).items():
    print(f"  {state}: {median:.3f}")
    
print(f"\nBottom 5 States by Median Income (MW):")
for state, median in state_medians.tail(5).items():
    print(f"  {state}: {median:.3f}")

print("\n" + "="*70 + "\n")

## 3. Income Mobility Analysis

Análise de mobilidade de renda simulada através de transições entre faixas de renda ao longo do tempo (usando dados de trimestres diferentes como proxy).

In [ ]:
# Income Mobility Analysis
# Note: This is a cross-sectional simulation since PNADC doesn't have panel data
# We'll use different quarters and age cohorts as a proxy for longitudinal analysis

def create_income_brackets(income_series, n_brackets=5):
    """Create income brackets based on percentiles."""
    percentiles = np.linspace(0, 100, n_brackets + 1)
    thresholds = np.percentile(income_series.dropna(), percentiles)
    
    def assign_bracket(income):
        if pd.isna(income) or income <= 0:
            return np.nan
        for i, threshold in enumerate(thresholds[1:], 1):
            if income <= threshold:
                return f'Q{i}'
        return f'Q{n_brackets}'
    
    return income_series.apply(assign_bracket), thresholds

# Create mobility analysis across quarters
print("=== INCOME MOBILITY ANALYSIS ===\n")

# 1. Create income quintiles for the entire dataset
df_mobility = df[df[mw_col].notna() & (df[mw_col] > 0)].copy()
df_mobility['income_quintile'], quintile_thresholds = create_income_brackets(df_mobility[mw_col], 5)

print("1. INCOME QUINTILE DEFINITIONS (MW)")
print("-" * 50)
for i, threshold in enumerate(quintile_thresholds[1:], 1):
    lower = quintile_thresholds[i-1] if i > 1 else 0
    print(f"Q{i}: {lower:.2f} - {threshold:.2f} MW")

# Distribution across quintiles
quintile_dist = df_mobility['income_quintile'].value_counts().sort_index()
print(f"\nDistribution across quintiles:")
for q, count in quintile_dist.items():
    pct = count / len(df_mobility) * 100
    print(f"{q}: {count:,} ({pct:.1f}%)")

print("\n" + "="*70 + "\n")

# 2. Simulate mobility across quarters (cross-sectional approximation)
# Group by UF and compare income distribution patterns across quarters

mobility_matrix = pd.DataFrame(index=[f'Q{i}' for i in range(1, 6)], 
                              columns=[f'Q{i}' for i in range(1, 6)], 
                              data=0.0)

# Create age-education cohorts for pseudo-panel analysis
df_mobility['age_cohort'] = pd.cut(df_mobility['V2009__idade_na_data_de_referncia'], 
                                   bins=[0, 25, 35, 45, 55, 65, 100],
                                   labels=['18-25', '26-35', '36-45', '46-55', '56-65', '65+'])

df_mobility['education_level'] = df_mobility['V3009A__curso_mais_elevado_que_frequentou'].astype(str)

# Calculate transition probabilities by comparing similar cohorts across quarters
transitions = []

for uf in df_mobility['UF_label'].unique():
    uf_data = df_mobility[df_mobility['UF_label'] == uf]
    
    for cohort in df_mobility['age_cohort'].unique():
        if pd.isna(cohort):
            continue
            
        cohort_data = uf_data[uf_data['age_cohort'] == cohort]
        
        if len(cohort_data) < 10:  # Skip small groups
            continue
            
        # Compare quintile distributions across available quarters
        quarters = cohort_data['Trimestre__trimestre_de_referncia'].unique()
        
        if len(quarters) > 1:
            # Calculate mobility between first and last available quarter
            q1_data = cohort_data[cohort_data['Trimestre__trimestre_de_referncia'] == quarters[0]]
            q2_data = cohort_data[cohort_data['Trimestre__trimestre_de_referncia'] == quarters[-1]]
            
            if len(q1_data) > 5 and len(q2_data) > 5:
                # Calculate quintile distributions
                q1_dist = q1_data['income_quintile'].value_counts(normalize=True)
                q2_dist = q2_data['income_quintile'].value_counts(normalize=True)
                
                transitions.append({
                    'uf': uf,
                    'cohort': cohort,
                    'q1_dist': q1_dist,
                    'q2_dist': q2_dist,
                    'sample_size': len(q1_data) + len(q2_data)
                })

print("2. PSEUDO-MOBILITY ANALYSIS")
print("-" * 50)
print(f"Analyzed {len(transitions)} UF-cohort combinations")

# Calculate average mobility patterns
if transitions:
    # Aggregate transition patterns
    avg_initial = pd.Series([0.0] * 5, index=[f'Q{i}' for i in range(1, 6)])
    avg_final = pd.Series([0.0] * 5, index=[f'Q{i}' for i in range(1, 6)])
    
    for t in transitions:
        weight = t['sample_size'] / sum([x['sample_size'] for x in transitions])
        
        for quintile in [f'Q{i}' for i in range(1, 6)]:
            avg_initial[quintile] += t['q1_dist'].get(quintile, 0) * weight
            avg_final[quintile] += t['q2_dist'].get(quintile, 0) * weight
    
    mobility_df = pd.DataFrame({
        'Initial Distribution': avg_initial * 100,
        'Final Distribution': avg_final * 100,
        'Change (pp)': (avg_final - avg_initial) * 100
    }).round(2)
    
    print("\nAggregated Mobility Patterns (Percentage Points):")
    display(mobility_df)
    
    # Calculate mobility indices
    upward_mobility = sum([max(0, change) for change in mobility_df['Change (pp)'].values])
    downward_mobility = sum([abs(min(0, change)) for change in mobility_df['Change (pp)'].values])
    net_mobility = upward_mobility - downward_mobility
    
    print(f"\nMobility Indices:")
    print(f"  Upward mobility: {upward_mobility:.2f} pp")
    print(f"  Downward mobility: {downward_mobility:.2f} pp")
    print(f"  Net mobility: {net_mobility:.2f} pp")
    
    # Persistence rates (percentage staying in same quintile)
    persistence = 100 - (upward_mobility + downward_mobility)
    print(f"  Income persistence: {persistence:.2f}%")

print("\n" + "="*70 + "\n")

# 3. Intergenerational mobility proxy using age and education
print("3. INTERGENERATIONAL MOBILITY ANALYSIS (CROSS-SECTIONAL)")
print("-" * 50)

# Use age and household composition as proxy for generations
# Use V2005_label for text values, or V2005__condio_no_domiclio == 1 for household head
df_mobility['is_household_head'] = df_mobility['V2005__condio_no_domiclio'] == 1  # Code 1 = Pessoa responsável

# Compare income by age groups within households
household_income_analysis = []

for dom_id in df_mobility['dom_id'].unique():
    household = df_mobility[df_mobility['dom_id'] == dom_id]
    
    if len(household) >= 2:  # Multi-person households
        # Get household head and others
        head = household[household['is_household_head'] == True]
        others = household[household['is_household_head'] == False]
        
        if len(head) > 0 and len(others) > 0:
            head_income = head[mw_col].mean()
            others_income = others[mw_col].mean()
            head_age = head['V2009__idade_na_data_de_referncia'].mean()
            others_age = others['V2009__idade_na_data_de_referncia'].mean()
            
            if pd.notna(head_income) and pd.notna(others_income) and head_income > 0 and others_income > 0:
                household_income_analysis.append({
                    'dom_id': dom_id,
                    'head_income': head_income,
                    'others_income': others_income,
                    'head_age': head_age,
                    'others_age': others_age,
                    'income_ratio': others_income / head_income,
                    'age_diff': head_age - others_age
                })

intergenerational_df = pd.DataFrame(household_income_analysis)

if len(intergenerational_df) > 0:
    print(f"Analyzed {len(intergenerational_df)} multi-person households")
    
    # Calculate correlation between head and others income
    income_correlation = intergenerational_df['head_income'].corr(intergenerational_df['others_income'])
    print(f"Income correlation (head vs others): {income_correlation:.4f}")
    
    # Income mobility by age difference
    intergenerational_df['age_gap_group'] = pd.cut(intergenerational_df['age_diff'], 
                                                  bins=[0, 15, 25, 35, 100],
                                                  labels=['0-15y', '15-25y', '25-35y', '35+y'])
    
    mobility_by_age_gap = intergenerational_df.groupby('age_gap_group').agg({
        'income_ratio': ['mean', 'median', 'std', 'count']
    }).round(3)
    
    print("\nIncome Ratio (Others/Head) by Age Gap:")
    display(mobility_by_age_gap)
    
    # Rank mobility analysis
    intergenerational_df['head_quintile'] = pd.qcut(intergenerational_df['head_income'], 5, labels=[f'Q{i}' for i in range(1, 6)])
    intergenerational_df['others_quintile'] = pd.qcut(intergenerational_df['others_income'], 5, labels=[f'Q{i}' for i in range(1, 6)])
    
    # Transition matrix
    transition_matrix = pd.crosstab(intergenerational_df['head_quintile'], 
                                   intergenerational_df['others_quintile'], 
                                   normalize='index') * 100
    
    print("\nIntergenerational Transition Matrix (Row %):")
    print("(Head Quintile → Others Quintile)")
    display(transition_matrix.round(1))
    
    # Calculate intergenerational mobility index
    diagonal_sum = sum([transition_matrix.loc[f'Q{i}', f'Q{i}'] for i in range(1, 6)])
    mobility_index = 100 - (diagonal_sum / 5)
    print(f"\nIntergenerational Mobility Index: {mobility_index:.2f}%")
    print("(Higher values indicate more mobility between generations)")

print("\n" + "="*70 + "\n")

## 4. Confidence Intervals for Mean Income by State

Cálculo de intervalos de confiança para renda média por estado usando bootstrap e métodos paramétricos.

In [ ]:
def bootstrap_confidence_interval(data, n_bootstrap=1000, confidence_level=0.95, statistic='mean'):
    """
    Calculate bootstrap confidence interval for a statistic.
    
    Args:
        data: Array of data points
        n_bootstrap: Number of bootstrap samples
        confidence_level: Confidence level (default 0.95)
        statistic: Statistic to calculate ('mean' or 'median')
    
    Returns:
        Dictionary with point estimate and confidence interval
    """
    data = np.array(data)
    data = data[~np.isnan(data)]
    
    if len(data) == 0:
        return {'point_estimate': np.nan, 'lower_ci': np.nan, 'upper_ci': np.nan}
    
    # Original statistic
    if statistic == 'mean':
        point_estimate = np.mean(data)
    else:
        point_estimate = np.median(data)
    
    # Bootstrap samples
    bootstrap_stats = []
    n = len(data)
    
    for _ in range(n_bootstrap):
        bootstrap_sample = np.random.choice(data, size=n, replace=True)
        if statistic == 'mean':
            bootstrap_stats.append(np.mean(bootstrap_sample))
        else:
            bootstrap_stats.append(np.median(bootstrap_sample))
    
    # Calculate confidence intervals
    alpha = 1 - confidence_level
    lower_percentile = (alpha / 2) * 100
    upper_percentile = (1 - alpha / 2) * 100
    
    lower_ci = np.percentile(bootstrap_stats, lower_percentile)
    upper_ci = np.percentile(bootstrap_stats, upper_percentile)
    
    return {
        'point_estimate': point_estimate,
        'lower_ci': lower_ci,
        'upper_ci': upper_ci,
        'ci_width': upper_ci - lower_ci,
        'margin_error': (upper_ci - lower_ci) / 2
    }

def parametric_confidence_interval(data, confidence_level=0.95):
    """
    Calculate parametric confidence interval assuming normal distribution.
    
    Args:
        data: Array of data points
        confidence_level: Confidence level (default 0.95)
    
    Returns:
        Dictionary with confidence interval
    """
    data = np.array(data)
    data = data[~np.isnan(data)]
    
    if len(data) <= 1:
        return {'point_estimate': np.nan, 'lower_ci': np.nan, 'upper_ci': np.nan}
    
    mean = np.mean(data)
    std_error = stats.sem(data)  # Standard error of the mean
    
    # t-distribution for small samples
    df = len(data) - 1
    alpha = 1 - confidence_level
    t_critical = stats.t.ppf(1 - alpha / 2, df)
    
    margin_error = t_critical * std_error
    
    return {
        'point_estimate': mean,
        'lower_ci': mean - margin_error,
        'upper_ci': mean + margin_error,
        'ci_width': 2 * margin_error,
        'margin_error': margin_error,
        'std_error': std_error,
        't_critical': t_critical
    }

# Calculate confidence intervals for mean income by state
print("=== CONFIDENCE INTERVALS FOR MEAN INCOME BY STATE ===\n")

# Filter states with sufficient sample size
state_sample_sizes = df_income['UF_label'].value_counts()
states_with_sufficient_data = state_sample_sizes[state_sample_sizes >= 100].index

print(f"Analyzing {len(states_with_sufficient_data)} states with ≥100 observations")
print(f"Total observations: {state_sample_sizes[states_with_sufficient_data].sum():,}")

# Calculate confidence intervals for each state
confidence_results = []

for state in states_with_sufficient_data:
    state_income = df_income[df_income['UF_label'] == state][mw_col]
    
    # Bootstrap CI (non-parametric)
    bootstrap_mean = bootstrap_confidence_interval(state_income, statistic='mean')
    bootstrap_median = bootstrap_confidence_interval(state_income, statistic='median')
    
    # Parametric CI (assuming normality)
    parametric_mean = parametric_confidence_interval(state_income)
    
    confidence_results.append({
        'State': state,
        'Sample_Size': len(state_income),
        'Mean': bootstrap_mean['point_estimate'],
        'Bootstrap_Mean_Lower': bootstrap_mean['lower_ci'],
        'Bootstrap_Mean_Upper': bootstrap_mean['upper_ci'],
        'Bootstrap_Mean_Width': bootstrap_mean['ci_width'],
        'Parametric_Mean_Lower': parametric_mean['lower_ci'],
        'Parametric_Mean_Upper': parametric_mean['upper_ci'],
        'Parametric_Mean_Width': parametric_mean['ci_width'],
        'Median': bootstrap_median['point_estimate'],
        'Bootstrap_Median_Lower': bootstrap_median['lower_ci'],
        'Bootstrap_Median_Upper': bootstrap_median['upper_ci'],
        'Standard_Error': parametric_mean['std_error']
    })

# Create results DataFrame
confidence_df = pd.DataFrame(confidence_results).round(4)
confidence_df = confidence_df.sort_values('Mean', ascending=False)

# Display top 10 states by mean income with confidence intervals
print("1. TOP 10 STATES BY MEAN INCOME (with 95% Confidence Intervals)")
print("-" * 80)

top10_df = confidence_df.head(10)[['State', 'Sample_Size', 'Mean', 
                                   'Bootstrap_Mean_Lower', 'Bootstrap_Mean_Upper', 
                                   'Bootstrap_Mean_Width']]

for _, row in top10_df.iterrows():
    print(f"{row['State']:25} | n={row['Sample_Size']:5} | "
          f"Mean: {row['Mean']:.3f} MW "
          f"[{row['Bootstrap_Mean_Lower']:.3f}, {row['Bootstrap_Mean_Upper']:.3f}] "
          f"(±{row['Bootstrap_Mean_Width']/2:.3f})")

print("\n" + "="*80 + "\n")

# Display bottom 10 states
print("2. BOTTOM 10 STATES BY MEAN INCOME (with 95% Confidence Intervals)")
print("-" * 80)

bottom10_df = confidence_df.tail(10)[['State', 'Sample_Size', 'Mean', 
                                      'Bootstrap_Mean_Lower', 'Bootstrap_Mean_Upper', 
                                      'Bootstrap_Mean_Width']]

for _, row in bottom10_df.iterrows():
    print(f"{row['State']:25} | n={row['Sample_Size']:5} | "
          f"Mean: {row['Mean']:.3f} MW "
          f"[{row['Bootstrap_Mean_Lower']:.3f}, {row['Bootstrap_Mean_Upper']:.3f}] "
          f"(±{row['Bootstrap_Mean_Width']/2:.3f})")

print("\n" + "="*80 + "\n")

# Compare bootstrap vs parametric confidence intervals
print("3. BOOTSTRAP vs PARAMETRIC CONFIDENCE INTERVALS COMPARISON")
print("-" * 80)

comparison_df = confidence_df[['State', 'Mean', 'Bootstrap_Mean_Width', 'Parametric_Mean_Width']].copy()
comparison_df['Width_Difference'] = comparison_df['Bootstrap_Mean_Width'] - comparison_df['Parametric_Mean_Width']
comparison_df['Width_Ratio'] = comparison_df['Bootstrap_Mean_Width'] / comparison_df['Parametric_Mean_Width']

# Show states with largest differences
print("States with largest differences in CI width (Bootstrap vs Parametric):")
extreme_diff = comparison_df.nlargest(5, 'Width_Difference')[['State', 'Bootstrap_Mean_Width', 'Parametric_Mean_Width', 'Width_Difference']]
display(extreme_diff.round(4))

print(f"\nAverage width ratio (Bootstrap/Parametric): {comparison_df['Width_Ratio'].mean():.3f}")
print(f"Bootstrap wider in {sum(comparison_df['Width_Ratio'] > 1)} of {len(comparison_df)} states")

print("\n" + "="*80 + "\n")

In [ ]:
# Visualize confidence intervals
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# 1. Top 15 states - Bootstrap confidence intervals
ax1 = axes[0, 0]
top15 = confidence_df.head(15)
y_pos = np.arange(len(top15))

ax1.errorbar(top15['Mean'], y_pos, 
            xerr=top15['Bootstrap_Mean_Width']/2,
            fmt='o', capsize=3, capthick=1, markersize=4)
ax1.set_yticks(y_pos)
ax1.set_yticklabels(top15['State'])
ax1.set_xlabel('Mean Income (MW)')
ax1.set_title('Top 15 States: Mean Income with 95% CI (Bootstrap)')
ax1.grid(True, alpha=0.3)
ax1.invert_yaxis()

# 2. Bottom 15 states - Bootstrap confidence intervals
ax2 = axes[0, 1]
bottom15 = confidence_df.tail(15)
y_pos_b = np.arange(len(bottom15))

ax2.errorbar(bottom15['Mean'], y_pos_b, 
            xerr=bottom15['Bootstrap_Mean_Width']/2,
            fmt='o', capsize=3, capthick=1, markersize=4, color='red')
ax2.set_yticks(y_pos_b)
ax2.set_yticklabels(bottom15['State'])
ax2.set_xlabel('Mean Income (MW)')
ax2.set_title('Bottom 15 States: Mean Income with 95% CI (Bootstrap)')
ax2.grid(True, alpha=0.3)
ax2.invert_yaxis()

# 3. Confidence interval width vs sample size
ax3 = axes[1, 0]
ax3.scatter(confidence_df['Sample_Size'], confidence_df['Bootstrap_Mean_Width'], 
           alpha=0.6, s=30)
ax3.set_xlabel('Sample Size')
ax3.set_ylabel('CI Width (MW)')
ax3.set_title('Confidence Interval Width vs Sample Size')
ax3.set_xscale('log')
ax3.grid(True, alpha=0.3)

# Add trend line
z = np.polyfit(np.log(confidence_df['Sample_Size']), confidence_df['Bootstrap_Mean_Width'], 1)
p = np.poly1d(z)
ax3.plot(confidence_df['Sample_Size'], p(np.log(confidence_df['Sample_Size'])), "r--", alpha=0.8)

# 4. Bootstrap vs Parametric CI comparison
ax4 = axes[1, 1]
ax4.scatter(confidence_df['Parametric_Mean_Width'], confidence_df['Bootstrap_Mean_Width'], 
           alpha=0.6, s=30)
ax4.plot([0, confidence_df['Parametric_Mean_Width'].max()], 
         [0, confidence_df['Parametric_Mean_Width'].max()], 'r--', alpha=0.8)
ax4.set_xlabel('Parametric CI Width (MW)')
ax4.set_ylabel('Bootstrap CI Width (MW)')
ax4.set_title('Bootstrap vs Parametric CI Width Comparison')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical significance testing between states
print("4. STATISTICAL SIGNIFICANCE TESTING BETWEEN STATES")
print("-" * 80)

# Test if top 3 states are significantly different from bottom 3 states
top3_states = confidence_df.head(3)['State'].values
bottom3_states = confidence_df.tail(3)['State'].values

print("Comparing TOP 3 vs BOTTOM 3 states:")
print(f"Top 3: {', '.join(top3_states)}")
print(f"Bottom 3: {', '.join(bottom3_states)}")

# Perform Mann-Whitney U test
top3_income = df_income[df_income['UF_label'].isin(top3_states)][mw_col]
bottom3_income = df_income[df_income['UF_label'].isin(bottom3_states)][mw_col]

mw_stat, mw_p = mannwhitneyu(top3_income, bottom3_income, alternative='two-sided')

print(f"\nMann-Whitney U Test:")
print(f"  Statistic: {mw_stat}")
print(f"  P-value: {mw_p:.2e}")
print(f"  Result: {'Significantly different' if mw_p < 0.05 else 'Not significantly different'}")

# Effect size (Cohen's d)
top3_mean = top3_income.mean()
bottom3_mean = bottom3_income.mean()
pooled_std = np.sqrt(((len(top3_income) - 1) * top3_income.var() + 
                     (len(bottom3_income) - 1) * bottom3_income.var()) / 
                    (len(top3_income) + len(bottom3_income) - 2))
cohens_d = (top3_mean - bottom3_mean) / pooled_std

print(f"\nEffect Size (Cohen's d): {cohens_d:.3f}")
magnitude = 'small' if abs(cohens_d) < 0.5 else 'medium' if abs(cohens_d) < 0.8 else 'large'
print(f"Magnitude: {magnitude}")

print(f"\nDescriptive Statistics:")
print(f"  Top 3 states mean: {top3_mean:.3f} MW (n={len(top3_income)})")
print(f"  Bottom 3 states mean: {bottom3_mean:.3f} MW (n={len(bottom3_income)})")
print(f"  Difference: {top3_mean - bottom3_mean:.3f} MW ({((top3_mean/bottom3_mean - 1)*100):+.1f}%)")

print("\n" + "="*80 + "\n")

## 5. Multivariate Analysis for Income Determinants

Análise multivariada para identificar os principais determinantes da renda, incluindo regressão múltipla, PCA e análise de clusters.

In [ ]:
import statsmodels.api as sm
# Prepare comprehensive dataset for multivariate analysis
print("=== MULTIVARIATE ANALYSIS FOR INCOME DETERMINANTS ===\n")

# Select relevant variables for analysis
multivar_columns = [
    # Demographics
    'V2007__sexo',  # Gender
    'V2009__idade_na_data_de_referncia',  # Age
    'V2010__cor_ou_raa',  # Race
    
    # Education
    'V3001__sabe_ler_e_escrever',  # Literacy
    'V3009A__curso_mais_elevado_que_frequentou',  # Education level
    
    # Geography
    'UF__unidade_da_federao',  # State
    'Capital__municpio_da_capital',  # Capital city
    
    # Household
    'VD2003__nmero_de_componentes_do_domic',  # Household size
    'V2005__condio_no_domiclio',  # Household position
    
    # Work-related (if available)
    'VD4001__condicao_de_forca_de_trabalho',  # Labor force status
    'VD4002__condicao_de_ocupacao',  # Employment status
]

# Keep only columns that exist in the dataset
# Filter to only existing columns
available_columns = [col for col in multivar_columns if col in df.columns]
analysis_df = df[available_columns + [mw_col]].copy()

# Clean data - remove rows with missing income
analysis_df = analysis_df[analysis_df[mw_col].notna() & (analysis_df[mw_col] > 0)]

print(f"Dataset for multivariate analysis:")
print(f"  Total observations: {len(analysis_df):,}")
print(f"  Available variables: {len(available_columns)}")
print(f"  Variables: {available_columns}")

print("\n" + "="*80 + "\n")

# 1. MULTIPLE LINEAR REGRESSION ANALYSIS
print("1. MULTIPLE LINEAR REGRESSION ANALYSIS")
print("-" * 50)

# Prepare features for regression
reg_df = analysis_df.copy()

# Create dummy variables for categorical features
categorical_features = []
numerical_features = ['V2009__idade_na_data_de_referncia', 'VD2003__nmero_de_componentes_do_domic']

for col in available_columns:
    if col not in numerical_features:
        # Create dummy variables
        dummies = pd.get_dummies(reg_df[col], prefix=col[:10], dummy_na=False)
        categorical_features.extend(dummies.columns.tolist())
        reg_df = pd.concat([reg_df, dummies], axis=1)

# Select features for regression (limit to avoid overfitting)
all_features = numerical_features + categorical_features
X_reg = reg_df[all_features].fillna(0)  # Fill any remaining NaN with 0
y_reg = reg_df[mw_col]

# Add constant for regression
X_reg_const = sm.add_constant(X_reg)

# Fit multiple regression model
try:
    reg_model = sm.OLS(y_reg, X_reg_const).fit()
    
    print(f"Regression Results:")
    print(f"  R-squared: {reg_model.rsquared:.4f}")
    print(f"  Adjusted R-squared: {reg_model.rsquared_adj:.4f}")
    print(f"  F-statistic: {reg_model.fvalue:.2f}")
    print(f"  F-statistic p-value: {reg_model.f_pvalue:.2e}")
    
    # Get top 10 most significant coefficients
    coeffs = pd.DataFrame({
        'Variable': reg_model.params.index,
        'Coefficient': reg_model.params.values,
        'P-value': reg_model.pvalues.values,
        'Significant': reg_model.pvalues.values < 0.05
    })
    coeffs = coeffs[coeffs['Variable'] != 'const']  # Remove intercept
    coeffs['Abs_Coefficient'] = abs(coeffs['Coefficient'])
    
    # Top 10 by absolute coefficient value
    print(f"\nTop 10 Variables by Coefficient Magnitude:")
    top_coeffs = coeffs.nlargest(10, 'Abs_Coefficient')[['Variable', 'Coefficient', 'P-value', 'Significant']]
    for _, row in top_coeffs.iterrows():
        sig_marker = "***" if row['P-value'] < 0.001 else "**" if row['P-value'] < 0.01 else "*" if row['P-value'] < 0.05 else ""
        print(f"  {row['Variable'][:30]:30} | Coeff: {row['Coefficient']:+7.4f} | p: {row['P-value']:.3e} {sig_marker}")
    
    # Model diagnostics
    print(f"\nModel Diagnostics:")
    print(f"  Number of observations: {int(reg_model.nobs)}")
    print(f"  Number of parameters: {len(reg_model.params)}")
    print(f"  Log-likelihood: {reg_model.llf:.2f}")
    print(f"  AIC: {reg_model.aic:.2f}")
    print(f"  BIC: {reg_model.bic:.2f}")
    
except Exception as e:
    print(f"Regression analysis failed: {e}")
    print("This may be due to multicollinearity or insufficient data.")

print("\n" + "="*80 + "\n")

In [ ]:
# 2. PRINCIPAL COMPONENT ANALYSIS (PCA)
print("2. PRINCIPAL COMPONENT ANALYSIS (PCA)")
print("-" * 50)

# Prepare data for PCA - use only numerical features and encoded categoricals
pca_df = analysis_df.copy()

# Encode categorical variables numerically
label_encoders_pca = {}
for col in available_columns:
    if col not in numerical_features:
        le = LabelEncoder()
        pca_df[col] = le.fit_transform(pca_df[col].astype(str))
        label_encoders_pca[col] = le

# Select features for PCA
pca_features = available_columns
X_pca = pca_df[pca_features].fillna(pca_df[pca_features].median())

# Standardize features
scaler_pca = StandardScaler()
X_pca_scaled = scaler_pca.fit_transform(X_pca)

# Perform PCA
pca = PCA()
X_pca_transformed = pca.fit_transform(X_pca_scaled)

# Calculate variance explained
variance_explained = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(variance_explained)

print(f"PCA Results:")
print(f"  Number of components: {len(variance_explained)}")
print(f"  Variance explained by first 5 components:")
for i in range(min(5, len(variance_explained))):
    print(f"    PC{i+1}: {variance_explained[i]:.4f} ({cumulative_variance[i]:.4f} cumulative)")

# Number of components for 80% and 90% variance
pc80 = np.argmax(cumulative_variance >= 0.80) + 1
pc90 = np.argmax(cumulative_variance >= 0.90) + 1
print(f"  Components for 80% variance: {pc80}")
print(f"  Components for 90% variance: {pc90}")

# Component loadings for first 3 PCs
print(f"\nComponent Loadings (First 3 PCs):")
loadings_df = pd.DataFrame(
    pca.components_[:3].T,
    columns=['PC1', 'PC2', 'PC3'],
    index=pca_features
)

# Show top loadings for each component
for pc in ['PC1', 'PC2', 'PC3']:
    print(f"\n{pc} - Top 5 positive and negative loadings:")
    sorted_loadings = loadings_df[pc].sort_values(key=abs, ascending=False)
    for var, loading in sorted_loadings.head(5).items():
        print(f"  {var[:25]:25} | {loading:+.4f}")

# Correlation between PCs and income
pc_income_corr = []
for i in range(min(10, len(variance_explained))):
    corr = np.corrcoef(X_pca_transformed[:, i], pca_df[mw_col])[0, 1]
    pc_income_corr.append(corr)

print(f"\nCorrelation between Principal Components and Income:")
for i, corr in enumerate(pc_income_corr):
    print(f"  PC{i+1}: {corr:+.4f}")

print("\n" + "="*80 + "\n")

# 3. CLUSTER ANALYSIS
print("3. CLUSTER ANALYSIS")
print("-" * 50)

# Use first few principal components for clustering
n_components_cluster = min(pc80, 10)  # Use components that explain 80% variance, max 10
X_cluster = X_pca_transformed[:, :n_components_cluster]

print(f"Using {n_components_cluster} principal components for clustering")

# Determine optimal number of clusters using elbow method
max_clusters = 10
inertias = []
silhouette_scores = []

for k in range(2, max_clusters + 1):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_cluster)
    inertias.append(kmeans.inertia_)
    
    # Calculate silhouette score
    from sklearn.metrics import silhouette_score
    silhouette = silhouette_score(X_cluster, cluster_labels)
    silhouette_scores.append(silhouette)

# Find optimal number of clusters
optimal_k = np.argmax(silhouette_scores) + 2  # +2 because range starts at 2

print(f"Optimal number of clusters (silhouette score): {optimal_k}")
print(f"Silhouette scores:")
for i, score in enumerate(silhouette_scores, 2):
    print(f"  k={i}: {score:.4f}")

# Perform final clustering
kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans_final.fit_predict(X_cluster)
pca_df['cluster'] = cluster_labels

# Analyze clusters
print(f"\nCluster Analysis Results:")
cluster_analysis = pca_df.groupby('cluster').agg({
    mw_col: ['count', 'mean', 'median', 'std'],
    'V2009__idade_na_data_de_referncia': 'mean',
    'VD2003__nmero_de_componentes_do_domic': 'mean'
}).round(3)

cluster_analysis.columns = ['Count', 'Income_Mean', 'Income_Median', 'Income_Std', 'Age_Mean', 'HH_Size_Mean']
display(cluster_analysis)

# Cluster characteristics
print(f"\nCluster Characteristics:")
for cluster_id in range(optimal_k):
    cluster_data = pca_df[pca_df['cluster'] == cluster_id]
    print(f"\nCluster {cluster_id} (n={len(cluster_data)}):")
    print(f"  Mean income: {cluster_data[mw_col].mean():.3f} MW")
    print(f"  Mean age: {cluster_data['V2009__idade_na_data_de_referncia'].mean():.1f} years")
    print(f"  Mean household size: {cluster_data['VD2003__nmero_de_componentes_do_domic'].mean():.1f}")
    
    # Most common categorical values in cluster
    for col in ['V2007__sexo', 'V2010__cor_ou_raa']:
        if col in available_columns:
            most_common = cluster_data[col].mode().iloc[0] if not cluster_data[col].empty else 'N/A'
            print(f"  Most common {col.split('__')[1]}: {most_common}")

print("\n" + "="*80 + "\n")

In [ ]:
# 4. VISUALIZATION OF MULTIVARIATE RESULTS
print("4. VISUALIZATION OF MULTIVARIATE RESULTS")
print("-" * 50)

fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# 1. PCA Scree Plot
ax1 = axes[0, 0]
ax1.plot(range(1, min(11, len(variance_explained) + 1)), variance_explained[:10], 'bo-', linewidth=2, markersize=6)
ax1.plot(range(1, min(11, len(variance_explained) + 1)), cumulative_variance[:10], 'ro-', linewidth=2, markersize=6)
ax1.set_xlabel('Principal Component')
ax1.set_ylabel('Proportion of Variance Explained')
ax1.set_title('PCA Scree Plot')
ax1.legend(['Individual', 'Cumulative'])
ax1.grid(True, alpha=0.3)
ax1.axhline(y=0.8, color='g', linestyle='--', alpha=0.7, label='80% line')

# 2. PC1 vs PC2 Scatter Plot colored by income
ax2 = axes[0, 1]
scatter = ax2.scatter(X_pca_transformed[:, 0], X_pca_transformed[:, 1], 
                     c=pca_df[mw_col], cmap='viridis', alpha=0.6, s=10)
ax2.set_xlabel(f'PC1 ({variance_explained[0]:.3f} variance)')
ax2.set_ylabel(f'PC2 ({variance_explained[1]:.3f} variance)')
ax2.set_title('PCA: PC1 vs PC2 (colored by income)')
plt.colorbar(scatter, ax=ax2, label='Income (MW)')

# 3. Cluster Analysis - PC1 vs PC2 colored by cluster
ax3 = axes[0, 2]
colors = plt.cm.Set3(np.linspace(0, 1, optimal_k))
for i in range(optimal_k):
    cluster_mask = pca_df['cluster'] == i
    ax3.scatter(X_pca_transformed[cluster_mask, 0], X_pca_transformed[cluster_mask, 1],
               c=[colors[i]], label=f'Cluster {i}', alpha=0.6, s=10)
ax3.set_xlabel(f'PC1 ({variance_explained[0]:.3f} variance)')
ax3.set_ylabel(f'PC2 ({variance_explained[1]:.3f} variance)')
ax3.set_title('Cluster Analysis: PC1 vs PC2')
ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# 4. Income distribution by cluster
ax4 = axes[1, 0]
cluster_income_data = [pca_df[pca_df['cluster'] == i][mw_col] for i in range(optimal_k)]
ax4.boxplot(cluster_income_data, labels=[f'C{i}' for i in range(optimal_k)])
ax4.set_xlabel('Cluster')
ax4.set_ylabel('Income (MW)')
ax4.set_title('Income Distribution by Cluster')
ax4.grid(True, alpha=0.3)

# 5. Elbow plot for clustering
ax5 = axes[1, 1]
k_range = range(2, max_clusters + 1)
ax5.plot(k_range, inertias, 'bo-', linewidth=2, markersize=6)
ax5.set_xlabel('Number of Clusters (k)')
ax5.set_ylabel('Within-cluster Sum of Squares (WCSS)')
ax5.set_title('Elbow Method for Optimal k')
ax5.grid(True, alpha=0.3)
ax5.axvline(x=optimal_k, color='r', linestyle='--', alpha=0.7, label=f'Optimal k={optimal_k}')
ax5.legend()

# 6. Silhouette scores
ax6 = axes[1, 2]
ax6.plot(k_range, silhouette_scores, 'go-', linewidth=2, markersize=6)
ax6.set_xlabel('Number of Clusters (k)')
ax6.set_ylabel('Silhouette Score')
ax6.set_title('Silhouette Analysis')
ax6.grid(True, alpha=0.3)
ax6.axvline(x=optimal_k, color='r', linestyle='--', alpha=0.7, label=f'Optimal k={optimal_k}')
ax6.legend()

plt.tight_layout()
plt.show()

print("\n" + "="*80 + "\n")

# 5. SUMMARY OF KEY FINDINGS
print("5. SUMMARY OF KEY INCOME DETERMINANTS")
print("-" * 50)

# Compile key findings from all analyses
key_findings = {
    'Inequality Metrics': {
        'Gini Coefficient (Individual)': f"{gini_individual_mw:.4f}",
        'Gini Coefficient (Household)': f"{gini_household_mw:.4f}",
        'Top 10% Income Share': f"{decile_shares[-1]:.1f}%",
        'Bottom 50% Income Share': f"{decile_shares[:5].sum():.1f}%"
    }
}

if 'reg_model' in locals():
    # Top significant predictors from regression
    significant_vars = coeffs[coeffs['Significant'] == True].nlargest(5, 'Abs_Coefficient')
    key_findings['Top Regression Predictors'] = {
        row['Variable'][:30]: f"{row['Coefficient']:+.4f}" 
        for _, row in significant_vars.iterrows()
    }
    key_findings['Model Performance'] = {
        'R-squared': f"{reg_model.rsquared:.4f}",
        'Adjusted R-squared': f"{reg_model.rsquared_adj:.4f}"
    }

# PCA findings
key_findings['Dimensionality Reduction'] = {
    'Components for 80% variance': f"{pc80}",
    'First PC variance explained': f"{variance_explained[0]:.4f}",
    'PC1-Income correlation': f"{pc_income_corr[0]:+.4f}"
}

# Cluster findings
key_findings['Population Segments'] = {
    'Optimal number of clusters': f"{optimal_k}",
    'Highest income cluster mean': f"{cluster_analysis['Income_Mean'].max():.3f} MW",
    'Lowest income cluster mean': f"{cluster_analysis['Income_Mean'].min():.3f} MW",
    'Income ratio (highest/lowest)': f"{cluster_analysis['Income_Mean'].max() / cluster_analysis['Income_Mean'].min():.2f}"
}

# Statistical test findings
key_findings['Statistical Significance'] = {
    'Gender income difference': f"{'Significant' if gender_tests['kruskal_wallis']['significant'] else 'Not significant'}",
    'Race income difference': f"{'Significant' if race_tests['kruskal_wallis']['significant'] else 'Not significant'}",
    'Age group income difference': f"{'Significant' if age_tests['kruskal_wallis']['significant'] else 'Not significant'}",
    'State income difference': f"{'Significant' if state_tests['kruskal_wallis']['significant'] else 'Not significant'}"
}

print("KEY FINDINGS SUMMARY:")
for category, findings in key_findings.items():
    print(f"\n{category}:")
    for key, value in findings.items():
        print(f"  • {key}: {value}")

print(f"\nDATA COVERAGE:")
print(f"  • Total individuals analyzed: {len(df_income):,}")
print(f"  • Total households analyzed: {len(household_income):,}")
print(f"  • States with sufficient data: {len(states_with_sufficient_data)}")
print(f"  • Analysis period: {df['ym'].min()} to {df['ym'].max()}")

print(f"\nRECOMMENDations FOR POLICY:")
print(f"  • High inequality (Gini > 0.5) suggests need for redistribution policies")
print(f"  • Significant demographic differences indicate targeted interventions needed")
print(f"  • Geographic disparities require regional development focus")
print(f"  • Multivariate analysis reveals key leverage points for income improvement")

print("\n" + "="*80 + "\n")
print("ADVANCED STATISTICAL ANALYSIS COMPLETED SUCCESSFULLY!")
print("="*80)

## 📊 Visualizações Avançadas - PNADC

Este conjunto de visualizações oferece uma análise completa dos dados de renda brasileiros da PNADC:

### 🗺️ **1. Mapa Coroplético Interativo**
- Visualização geográfica da renda mediana por estado
- Cores representam diferentes faixas de renda
- Hover interativo com detalhes por estado
- Compatível com dados geográficos do IBGE

### 👥 **2. Pirâmides Demográficas com Sobreposição de Renda**
- Distribuição populacional por idade e sexo
- Camadas de cor representando faixas de renda
- Identificação de padrões demográficos e econômicos
- Comparação entre homens e mulheres

### 📈 **3. Séries Temporais de Renda**
- Evolução da renda ao longo dos trimestres
- Linhas de mediana e média com bandas de confiança
- Identificação de tendências e sazonalidades
- Anotações automáticas para insights chave

### 🌊 **4. Diagrama Sankey de Fluxo de Renda**
- Fluxo entre educação → idade → faixa de renda
- Identificação de caminhos de mobilidade social
- Cores categóricas para diferentes grupos
- Espessura dos fluxos representa volume populacional

### 🔥 **5. Heatmaps de Correlação Avançados**
- Matriz de correlação entre variáveis principais
- Ranking de correlações com renda
- Perfil demográfico por estado
- Versões estáticas (matplotlib/seaborn) e interativas (plotly)

### 🎛️ **6. Dashboard Interativo Resumo**
- Combinação de múltiplas visualizações em uma tela
- Comparações rápidas entre estados
- Distribuições e correlações lado a lado
- Layout responsivo e profissional

---

### 🎨 **Características Técnicas:**
- **Paletas de cores**: Adequadas para dados de renda (RdYlGn, viridis, RdYlBu)
- **Interatividade**: Hover, zoom, seleção em gráficos Plotly
- **Responsividade**: Layouts que se adaptam a diferentes tamanhos
- **Acessibilidade**: Contrastes adequados e labels em português
- **Performance**: Otimizadas para datasets grandes

### 🚀 **Como usar:**
1. Execute as células sequencialmente
2. As visualizações aparecerão automaticamente
3. Gráficos interativos permitem exploração detalhada
4. Dados processados ficam disponíveis para análises adicionais

---
*💡 **Dica**: Para melhor experiência, execute em um ambiente com boa conexão para carregar dados geográficos e bibliotecas de visualização.*

In [ ]:
# 6. Bonus: Advanced Interactive Dashboard Summary
def create_dashboard_summary(df, state_income_summary, corr_matrix):
    """
    Create a comprehensive dashboard summary combining all visualizations.
    """
    # Create subplots for dashboard
    fig = make_subplots(
        rows=2, cols=3,
        subplot_titles=(
            'Distribuição de Renda por Estado',
            'Renda por Faixa Etária e Sexo', 
            'Correlações com Renda',
            'Renda Mediana vs Média',
            'Distribuição de Renda',
            'Top Estados por Renda'
        ),
        specs=[[{"type": "bar"}, {"type": "box"}, {"type": "scatter"}],
               [{"type": "scatter"}, {"type": "histogram"}, {"type": "bar"}]],
        vertical_spacing=0.15,
        horizontal_spacing=0.1
    )
    
    # 1. Top states by income
    top_states = state_income_summary.nlargest(10, 'Renda_Mediana_SM')
    fig.add_trace(
        go.Bar(x=top_states['Renda_Mediana_SM'], 
               y=top_states['Estado'], 
               orientation='h',
               marker_color='lightblue',
               name='Renda Mediana'),
        row=1, col=1
    )
    
    # 2. Income by age and sex
    if 'V2007' in df.columns:
        age_groups = pd.cut(df['V2009'], bins=[0, 25, 35, 45, 55, 100], 
                           labels=['18-24', '25-34', '35-44', '45-54', '55+'])
        for sex in df['V2007'].unique()[:2]:  # Limit to 2 categories
            sex_data = df[df['V2007'] == sex]
            fig.add_trace(
                go.Box(y=sex_data[INCOME_COL], 
                       name=f'{sex}',
                       boxpoints='outliers'),
                row=1, col=2
            )
    
    # 3. Correlation with income
    if corr_matrix is not None:
        income_correlations = corr_matrix[INCOME_COL].abs().sort_values(ascending=False)[1:6]
        fig.add_trace(
            go.Scatter(x=list(range(len(income_correlations))), 
                      y=income_correlations.values,
                      mode='markers+lines',
                      marker=dict(size=10, color='red'),
                      name='Correlação'),
            row=1, col=3
        )
    
    # 4. Median vs Mean income scatter
    fig.add_trace(
        go.Scatter(x=state_income_summary['Renda_Mediana_SM'], 
                  y=state_income_summary['Renda_Media_SM'],
                  mode='markers+text',
                  text=state_income_summary['Estado'],
                  textposition='top center',
                  marker=dict(size=8, color='green'),
                  name='Estados'),
        row=2, col=1
    )
    
    # 5. Income distribution
    fig.add_trace(
        go.Histogram(x=df[INCOME_COL], 
                    nbinsx=30,
                    marker_color='orange',
                    name='Distribuição'),
        row=2, col=2
    )
    
    # 6. Bottom states for comparison
    bottom_states = state_income_summary.nsmallest(5, 'Renda_Mediana_SM')
    fig.add_trace(
        go.Bar(x=bottom_states['Renda_Mediana_SM'], 
               y=bottom_states['Estado'], 
               orientation='h',
               marker_color='lightcoral',
               name='Menores Rendas'),
        row=2, col=3
    )
    
    # Update layout
    fig.update_layout(
        height=800,
        showlegend=False,
        title_text="Dashboard PNADC - Análise de Renda Brasileira",
        title_x=0.5,
        title_font_size=18,
        font_family="Arial"
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Renda (SM)", row=1, col=1)
    fig.update_xaxes(title_text="Renda (SM)", row=2, col=1)
    fig.update_yaxes(title_text="Renda (SM)", row=2, col=1)
    fig.update_xaxes(title_text="Renda (SM)", row=2, col=2)
    
    return fig

# Create dashboard summary
try:
    fig_dashboard = create_dashboard_summary(df, state_income_summary, corr_matrix)
    fig_dashboard.show()
    print("✅ Dashboard interativo criado com sucesso!")
except Exception as e:
    print(f"⚠️  Erro ao criar dashboard: {e}")
    print("Isso pode acontecer se algumas variáveis não estiverem disponíveis no dataset.")

In [ ]:
# 5. Advanced Heatmaps for Correlation Analysis
def create_income_correlation_heatmaps(df):
    """
    Create advanced heatmap visualizations for income correlation analysis.
    """
    # Prepare numeric variables for correlation analysis
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Remove ID columns and irrelevant ones
    exclude_patterns = ['id', 'ID', 'codigo', 'sequencial', 'dom_id']
    numeric_cols = [col for col in numeric_cols if not any(pattern in col for pattern in exclude_patterns)]
    
    # Ensure we have the income column
    if INCOME_COL not in numeric_cols:
        print(f"Warning: {INCOME_COL} not found in numeric columns")
        return None, None
    
    # Limit to top correlated variables with income for readability
    correlations_with_income = df[numeric_cols].corrwith(df[INCOME_COL]).abs().sort_values(ascending=False)
    top_corr_vars = correlations_with_income.head(15).index.tolist()
    
    print("🔍 Variáveis mais correlacionadas com renda:")
    for var in top_corr_vars[:10]:
        corr_value = correlations_with_income[var]
        print(f"  {var}: {corr_value:.3f}")
    
    # Create correlation matrix
    correlation_matrix = df[top_corr_vars].corr()
    
    # 1. Basic Correlation Heatmap
    fig1, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    
    # Correlation heatmap
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix, 
                mask=mask,
                annot=True, 
                cmap='RdYlBu_r', 
                center=0,
                square=True,
                fmt='.2f',
                cbar_kws={"shrink": .8},
                ax=ax1)
    ax1.set_title('Matriz de Correlação - Variáveis vs Renda', fontsize=14, pad=20)
    ax1.tick_params(axis='x', rotation=45)
    ax1.tick_params(axis='y', rotation=0)
    
    # Income correlation bar plot
    income_correlations = correlations_with_income.head(10)
    colors = plt.cm.RdYlBu_r(np.linspace(0, 1, len(income_correlations)))
    bars = ax2.barh(range(len(income_correlations)), income_correlations.values, color=colors)
    ax2.set_yticks(range(len(income_correlations)))
    ax2.set_yticklabels([col.replace('__', '\n').replace('_', ' ') for col in income_correlations.index])
    ax2.set_xlabel('Correlação Absoluta com Renda')
    ax2.set_title('Correlações Mais Fortes com Renda', fontsize=14, pad=20)
    ax2.grid(axis='x', alpha=0.3)
    
    # Add values on bars
    for i, bar in enumerate(bars):
        width = bar.get_width()
        ax2.text(width + 0.01, bar.get_y() + bar.get_height()/2, 
                f'{width:.3f}', ha='left', va='center')
    
    plt.tight_layout()
    plt.show()
    
    # 2. Interactive Plotly Heatmap
    fig2 = go.Figure(data=go.Heatmap(
        z=correlation_matrix.values,
        x=[col.replace('__', '<br>').replace('_', ' ') for col in correlation_matrix.columns],
        y=[col.replace('__', '<br>').replace('_', ' ') for col in correlation_matrix.index],
        colorscale='RdYlBu',
        zmid=0,
        text=correlation_matrix.values,
        texttemplate="%{text:.2f}",
        textfont={"size": 10},
        hoverongaps=False,
        hovertemplate='<b>%{y}</b><br><b>%{x}</b><br>Correlação: %{z:.3f}<extra></extra>'
    ))
    
    fig2.update_layout(
        title='Matriz de Correlação Interativa - Variáveis PNADC',
        title_x=0.5,
        font_family="Arial",
        height=700,
        width=800
    )
    
    fig2.show()
    
    # 3. State-Income Heatmap
    state_demographics = df.groupby('UF_label').agg({
        INCOME_COL: ['mean', 'median'],
        'V2009': 'mean',  # Average age
        'V2007': lambda x: (x == 'Mulher').mean() if 'V2007' in df.columns else 0.5,  # % women
    }).round(3)
    
    state_demographics.columns = ['Renda_Media', 'Renda_Mediana', 'Idade_Media', 'Perc_Mulheres']
    state_demographics = state_demographics.reset_index()
    
    # Pivot for heatmap
    state_metrics = state_demographics.set_index('UF_label')
    
    # Normalize for better visualization
    state_metrics_norm = (state_metrics - state_metrics.min()) / (state_metrics.max() - state_metrics.min())
    
    fig3 = plt.figure(figsize=(12, 10))
    sns.heatmap(state_metrics_norm.T, 
                annot=state_metrics.T, 
                fmt='.2f',
                cmap='viridis',
                cbar_kws={'label': 'Valores Normalizados'},
                square=False)
    plt.title('Perfil Demográfico e de Renda por Estado (PNADC)', fontsize=14, pad=20)
    plt.xlabel('Estados')
    plt.ylabel('Indicadores')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    return correlation_matrix, state_demographics

# Create correlation heatmaps
corr_matrix, state_demo = create_income_correlation_heatmaps(df)
print("\n✅ Heatmaps de correlação criados com sucesso!")

In [ ]:
# 4. Sankey Diagram: Income Flow Between Brackets
def create_income_sankey_diagram(df):
    """
    Create a Sankey diagram showing income flow between demographic groups and income brackets.
    """
    # Create age groups and income brackets
    df_viz = df.copy()
    df_viz['Idade_Grupo'] = pd.cut(df_viz['V2009'], 
                                   bins=[0, 25, 35, 45, 55, 100], 
                                   labels=['Jovens (0-24)', 'Adultos Jovens (25-34)', 
                                          'Adultos (35-44)', 'Maduros (45-54)', 'Seniores (55+)'])
    
    # Create income brackets based on quantiles
    income_quantiles = df_viz[INCOME_COL].quantile([0.2, 0.4, 0.6, 0.8]).values
    df_viz['Faixa_Renda'] = pd.cut(df_viz[INCOME_COL], 
                                   bins=[0] + list(income_quantiles) + [df_viz[INCOME_COL].max()],
                                   labels=['Muito Baixa', 'Baixa', 'Média', 'Alta', 'Muito Alta'])
    
    # Prepare education levels (assuming VD3006 is education)
    education_cols = [col for col in df.columns if 'educ' in col.lower() or 'VD3006' in col]
    if education_cols:
        df_viz['Educacao_Nivel'] = df_viz[education_cols[0]]
        # Simplify education levels for clarity
        education_mapping = {1: 'Sem Instrução', 2: 'Fundamental', 3: 'Médio', 4: 'Superior', 5: 'Pós-Graduação'}
        if df_viz['Educacao_Nivel'].dtype in ['int64', 'float64']:
            df_viz['Educacao_Nivel'] = df_viz['Educacao_Nivel'].fillna(1).astype(int).map(education_mapping).fillna('Não Informado')
    else:
        # Create synthetic education data
        df_viz['Educacao_Nivel'] = np.random.choice(['Fundamental', 'Médio', 'Superior'], len(df_viz), 
                                                   p=[0.4, 0.4, 0.2])
    
    # Create flow data from education to age group to income
    flow_data = []
    
    # Education -> Age Group flows
    edu_age_flows = df_viz.groupby(['Educacao_Nivel', 'Idade_Grupo']).size().reset_index(name='count')
    for _, row in edu_age_flows.iterrows():
        flow_data.append({
            'source': row['Educacao_Nivel'],
            'target': row['Idade_Grupo'], 
            'value': row['count']
        })
    
    # Age Group -> Income flows
    age_income_flows = df_viz.groupby(['Idade_Grupo', 'Faixa_Renda']).size().reset_index(name='count')
    for _, row in age_income_flows.iterrows():
        flow_data.append({
            'source': row['Idade_Grupo'],
            'target': row['Faixa_Renda'],
            'value': row['count']
        })
    
    # Create unique node list
    all_nodes = list(set([d['source'] for d in flow_data] + [d['target'] for d in flow_data]))
    node_indices = {node: i for i, node in enumerate(all_nodes)}
    
    # Prepare data for Sankey
    source_indices = [node_indices[d['source']] for d in flow_data]
    target_indices = [node_indices[d['target']] for d in flow_data]
    values = [d['value'] for d in flow_data]
    
    # Define colors for different categories
    colors = {
        # Education colors (blues)
        'Sem Instrução': '#08306b', 'Fundamental': '#08519c', 'Médio': '#3182bd', 'Superior': '#6baed6',
        # Age group colors (greens)
        'Jovens (0-24)': '#00441b', 'Adultos Jovens (25-34)': '#006d2c', 'Adultos (35-44)': '#31a354',
        'Maduros (45-54)': '#74c476', 'Seniores (55+)': '#bae4b3',
        # Income colors (reds)
        'Muito Baixa': '#67000d', 'Baixa': '#a50f15', 'Média': '#cb181d', 'Alta': '#fb6a4a', 'Muito Alta': '#fcae91'
    }
    
    node_colors = [colors.get(node, '#cccccc') for node in all_nodes]
    link_colors = ['rgba(135,206,250,0.4)' for _ in flow_data]
    
    # Create Sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=all_nodes,
            color=node_colors
        ),
        link=dict(
            source=source_indices,
            target=target_indices,
            value=values,
            color=link_colors
        )
    )])
    
    fig.update_layout(
        title_text="Fluxo de Renda: Educação → Idade → Faixa de Renda (PNADC)",
        title_x=0.5,
        font_size=12,
        height=600,
        font_family="Arial"
    )
    
    return fig, flow_data

# Create Sankey diagram
fig_sankey, sankey_data = create_income_sankey_diagram(df)
fig_sankey.show()

print(f"\n🌊 Fluxos de renda identificados: {len(sankey_data)} conexões")

In [ ]:
# 3. Time Series Visualization of Income Trends by Quarter
def create_income_time_series(df):
    """
    Create time series visualization showing income trends by quarter.
    Note: This assumes your data contains temporal information.
    """
    # Check if we have temporal data
    temporal_cols = [col for col in df.columns if 'trim' in col.lower() or 'quarter' in col.lower() or 'ano' in col.lower()]
    
    if not temporal_cols:
        print("⚠️  No temporal columns found. Creating a synthetic time series based on available data.")
        # Create synthetic quarterly data based on income distribution
        df_viz = df.copy()
        df_viz['Trimestre_Simulado'] = np.random.choice(['2023-Q1', '2023-Q2', '2023-Q3', '2023-Q4'], len(df))
    else:
        # Use actual temporal data if available
        df_viz = df.copy()
        print(f"📅 Found temporal columns: {temporal_cols}")
        # Use the first temporal column found
        df_viz['Trimestre'] = df_viz[temporal_cols[0]]
    
    # Calculate quarterly statistics
    quarter_col = 'Trimestre_Simulado' if not temporal_cols else 'Trimestre'
    quarterly_stats = df_viz.groupby(quarter_col)[INCOME_COL].agg([
        'mean', 'median', 'std', 'count',
        lambda x: x.quantile(0.25),
        lambda x: x.quantile(0.75)
    ]).reset_index()
    
    quarterly_stats.columns = [quarter_col, 'Media', 'Mediana', 'DesvPadrao', 'Observacoes', 'Q1', 'Q3']
    
    # Create the time series plot
    fig = go.Figure()
    
    # Add median line
    fig.add_trace(go.Scatter(
        x=quarterly_stats[quarter_col],
        y=quarterly_stats['Mediana'],
        mode='lines+markers',
        name='Renda Mediana',
        line=dict(color='#2E86AB', width=3),
        marker=dict(size=8)
    ))
    
    # Add mean line
    fig.add_trace(go.Scatter(
        x=quarterly_stats[quarter_col],
        y=quarterly_stats['Media'],
        mode='lines+markers',
        name='Renda Média',
        line=dict(color='#A23B72', width=3, dash='dash'),
        marker=dict(size=8)
    ))
    
    # Add confidence bands (Q1 to Q3)
    fig.add_trace(go.Scatter(
        x=quarterly_stats[quarter_col].tolist() + quarterly_stats[quarter_col].tolist()[::-1],
        y=quarterly_stats['Q1'].tolist() + quarterly_stats['Q3'].tolist()[::-1],
        fill='tonexty',
        fillcolor='rgba(46,134,171,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=True,
        name='Faixa Interquartil'
    ))
    
    fig.update_layout(
        title='Evolução da Renda ao Longo dos Trimestres (PNADC)',
        title_x=0.5,
        title_font_size=16,
        xaxis_title='Trimestre',
        yaxis_title='Renda (Salários Mínimos)',
        height=500,
        font_family="Arial",
        hovermode='x unified',
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )
    
    # Add annotations for key insights
    max_median_idx = quarterly_stats['Mediana'].idxmax()
    fig.add_annotation(
        x=quarterly_stats.iloc[max_median_idx][quarter_col],
        y=quarterly_stats.iloc[max_median_idx]['Mediana'],
        text=f"Maior mediana<br>{quarterly_stats.iloc[max_median_idx]['Mediana']:.2f} SM",
        showarrow=True,
        arrowhead=2,
        arrowcolor="#2E86AB",
        bgcolor="white",
        bordercolor="#2E86AB"
    )
    
    return fig, quarterly_stats

# Create time series visualization
fig_timeseries, quarterly_data = create_income_time_series(df)
fig_timeseries.show()

print("\n📈 Estatísticas trimestrais:")
print(quarterly_data)

In [ ]:
# 2. Demographic Pyramids with Income Overlays
def create_income_demographic_pyramids(df):
    """
    Create demographic pyramids showing population distribution by age and sex,
    with income overlays using color coding.
    """
    # Prepare age groups and income brackets
    df_viz = df.copy()
    df_viz['Idade_Grupo'] = pd.cut(df_viz['V2009'], 
                                   bins=[0, 18, 25, 35, 45, 55, 65, 100], 
                                   labels=['0-17', '18-24', '25-34', '35-44', '45-54', '55-64', '65+'])
    
    # Create income brackets
    income_percentiles = df_viz[INCOME_COL].quantile([0.25, 0.5, 0.75, 0.9]).values
    df_viz['Faixa_Renda'] = pd.cut(df_viz[INCOME_COL], 
                                   bins=[0] + list(income_percentiles) + [df_viz[INCOME_COL].max()],
                                   labels=['Baixa (Q1)', 'Média-Baixa (Q2)', 'Média (Q3)', 'Média-Alta (Q4)', 'Alta (Q5)'])
    
    # Group data for pyramid
    pyramid_data = df_viz.groupby(['Idade_Grupo', 'V2007', 'Faixa_Renda']).size().reset_index(name='Count')
    pyramid_data = pyramid_data.pivot_table(index=['Idade_Grupo', 'Faixa_Renda'], 
                                           columns='V2007', 
                                           values='Count', 
                                           fill_value=0).reset_index()
    
    # Create the demographic pyramid
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Homens', 'Mulheres'),
        shared_yaxes=True,
        horizontal_spacing=0.05
    )
    
    colors = px.colors.qualitative.Set3[:5]  # Colors for income brackets
    
    for i, income_bracket in enumerate(['Baixa (Q1)', 'Média-Baixa (Q2)', 'Média (Q3)', 'Média-Alta (Q4)', 'Alta (Q5)']):
        subset = pyramid_data[pyramid_data['Faixa_Renda'] == income_bracket]
        
        # Men (left side, negative values)
        fig.add_trace(
            go.Bar(
                x=-subset['Homem'],
                y=subset['Idade_Grupo'],
                name=income_bracket,
                orientation='h',
                marker_color=colors[i],
                hovertemplate='%{y}<br>Homens: %{x:,.0f}<extra></extra>',
                showlegend=(i == 0)
            ),
            row=1, col=1
        )
        
        # Women (right side, positive values)
        fig.add_trace(
            go.Bar(
                x=subset['Mulher'],
                y=subset['Idade_Grupo'],
                name=income_bracket,
                orientation='h',
                marker_color=colors[i],
                hovertemplate='%{y}<br>Mulheres: %{x:,.0f}<extra></extra>',
                showlegend=False
            ),
            row=1, col=2
        )
    
    fig.update_layout(
        title='Pirâmide Demográfica com Sobreposição de Renda (PNADC)',
        title_x=0.5,
        title_font_size=16,
        height=500,
        bargap=0.1,
        font_family="Arial",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    fig.update_xaxes(title_text="População", row=1, col=1)
    fig.update_xaxes(title_text="População", row=1, col=2)
    fig.update_yaxes(title_text="Grupos de Idade", row=1, col=1)
    
    return fig

# Create demographic pyramids
fig_pyramid = create_income_demographic_pyramids(df)
fig_pyramid.show()

In [ ]:
# 1. Interactive Choropleth Map: Median Income by State
def create_income_choropleth(df, brazil_geojson=None):
    """
    Create an interactive choropleth map showing median income by Brazilian state.
    """
    # Calculate median income by state
    state_income = df.groupby('UF_label')[INCOME_COL].agg(['median', 'mean', 'count']).reset_index()
    state_income.columns = ['Estado', 'Renda_Mediana_SM', 'Renda_Media_SM', 'Observacoes']
    
    # Convert to minimum wage values for better interpretation
    state_income['Renda_Mediana_Reais'] = state_income['Renda_Mediana_SM'] * 1320  # Approximate 2023 minimum wage
    state_income['Renda_Media_Reais'] = state_income['Renda_Media_SM'] * 1320
    
    print("📊 Renda mediana por estado (em salários mínimos):")
    print(state_income.sort_values('Renda_Mediana_SM', ascending=False).head(10))
    
    if brazil_geojson:
        # Create choropleth with real geographical data
        fig = px.choropleth(
            state_income,
            geojson=brazil_geojson,
            locations='Estado',
            color='Renda_Mediana_SM',
            hover_name='Estado',
            hover_data={
                'Renda_Mediana_SM': ':.2f',
                'Renda_Media_SM': ':.2f',
                'Renda_Mediana_Reais': ':,.0f',
                'Observacoes': ':,d'
            },
            color_continuous_scale='RdYlGn',
            title='Renda Mediana por Estado Brasileiro (PNADC)',
            labels={
                'Renda_Mediana_SM': 'Renda Mediana (SM)',
                'Estado': 'Estado'
            }
        )
    else:
        # Create a simple bar chart as fallback
        fig = px.bar(
            state_income.sort_values('Renda_Mediana_SM', ascending=True),
            x='Renda_Mediana_SM',
            y='Estado',
            orientation='h',
            color='Renda_Mediana_SM',
            color_continuous_scale='RdYlGn',
            title='Renda Mediana por Estado Brasileiro (PNADC)',
            labels={'Renda_Mediana_SM': 'Renda Mediana (Salários Mínimos)'},
            hover_data=['Renda_Media_SM', 'Observacoes']
        )
    
    fig.update_layout(
        title_font_size=16,
        title_x=0.5,
        height=600,
        font_family="Arial",
        coloraxis_colorbar_title="Salários<br>Mínimos"
    )
    
    return fig, state_income

# Create the choropleth map
fig_choropleth, state_income_summary = create_income_choropleth(df, brazil_geojson)
fig_choropleth.show()

In [ ]:
# Fetch Brazil States GeoJSON for Choropleth Map
import requests
import json

def fetch_brazil_geojson():
    """Fetch Brazil states GeoJSON data from a reliable source."""
    url = "https://raw.githubusercontent.com/holtzy/D3-graph-gallery/master/DATA/world.geojson"
    try:
        # Try to get Brazil-specific GeoJSON
        brazil_url = "https://servicodados.ibge.gov.br/api/v3/malhas/paises/BR?formato=application/vnd.geo+json&qualidade=maxima&intrarregiao=uf"
        response = requests.get(brazil_url, timeout=10)
        if response.status_code == 200:
            return response.json()
        else:
            # Fallback to a simpler Brazil GeoJSON
            fallback_url = "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson"
            response = requests.get(fallback_url, timeout=10)
            if response.status_code == 200:
                return response.json()
    except:
        pass
    
    # If all else fails, create a simple mock structure
    print("Warning: Using simplified state boundaries. Consider downloading proper GeoJSON data.")
    return None

# Fetch the GeoJSON data
brazil_geojson = fetch_brazil_geojson()
print("✅ Brazil GeoJSON data loaded successfully!" if brazil_geojson else "⚠️  Using fallback for map data")

In [ ]:
# Advanced Data Visualization Setup
# Install additional required packages for professional visualizations
import subprocess
import sys

# Install additional visualization packages if not present
packages_to_install = [
    'plotly>=5.0',
    'altair>=5.0', 
    'geopandas>=0.14.0',
    'folium>=0.15.0',
    'requests>=2.25.0'
]

for package in packages_to_install:
    try:
        __import__(package.split('>=')[0])
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])

# Import visualization libraries
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set_style("whitegrid")
alt.data_transformers.enable('json')

print("✅ Visualization libraries loaded successfully!")